In [8]:
import os 
import numpy as np
import pandas as pd
import ast
import glob
import shutil
import datetime


In [9]:
CROP_IMAGE_DIR = os.getenv('CROP_IMAGE_DIR_PATH')
test_data_id = '09'
metadata_name = 'meta_data/TWB_ABD_expand_modified_gasex_21072022.csv'
test_data_list_name = 'fattyliver_2_class_certained_0_123_4_20_40_dataset_lists/dataset'+str(test_data_id)+'/test_dataset'+str(test_data_id)+'.csv'
test_data_list = pd.read_csv(test_data_list_name)
meta_data = pd.read_csv(metadata_name, sep=",")

ground_truth_pos_mi_ids = [mi_id
         for mi_id in test_data_list['MI_ID'] 
         if meta_data[meta_data['MI_ID']==mi_id]['liver_fatty'].to_list()[0]  > 0 ]
selected_mi_ids = [mi_id for mi_id in ground_truth_pos_mi_ids
                   if len(ast.literal_eval(meta_data[meta_data['MI_ID']==mi_id]['IMG_ID_LIST'].to_list()[0])) >= 20
                   ]


In [4]:
result_dir = "/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/"
current_timestamp = datetime.datetime.now().strftime('%m-%d-%Y-%H-%M-%S')
all_subj_save_dir = os.path.join("/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results",f"ridge-{current_timestamp}")
all_subj_save_dir = "/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/ridge-08-06-2024-01-24-40"
if not os.path.exists(all_subj_save_dir):
    os.mkdir(all_subj_save_dir)
if result_dir.endswith("/"):
    csv_paths = glob.glob(result_dir+ "*/pred_results.csv")
else:
    csv_paths = glob.glob(result_dir+ "/*/pred_results.csv")
    
for i in csv_paths:
    df = pd.read_csv(i)
    if len(df.columns) < 15:
        print(i)

/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0018125/pred_results.csv
/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0018179/pred_results.csv
/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0015277/pred_results.csv
/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0020332/pred_results.csv
/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0020297/pred_results.csv
/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0020679/pred_results.csv
/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0022026/pred_results.csv
/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0006148/pred_results.csv
/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/07-12-2024-03-57-58/P0014694/pred_results.csv
/home/liuusa_tw/twbabd_image

In [10]:
result_dirs = ["/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/TEMP-elastic-net-08-02-2024-00-32-01",
               "/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/P0012750-RIDGE-RESULTS",
               "/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/correlation-old-dataset-08-01-2024-03-37-02"]

result_dirs = ["/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/elastic-net-old-dataset-08-01-2024-06-59-39",
               "/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/ridge-08-06-2024-01-24-40",
               "/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/correlation-old-dataset-08-01-2024-03-37-02"]

save_dir = "/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/test_organized_output"
current_timestamp = datetime.datetime.now().strftime('%m-%d-%Y-%H-%M-%S')
save_dir = os.path.join("/home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/", f"organized-output-{current_timestamp}")
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
else:
    print(f"{save_dir} already exists, files may be overwritten")
    
result_dict = {mi_id: {"csv_paths": [],
                       "plot_paths": [],} 
    for mi_id in selected_mi_ids}
for result_dir in result_dirs:
    completed_subj = [f.name for f in os.scandir(result_dir) if f.is_dir()]
    for mi_id in completed_subj:
        # if mi_id != "P0012750":
        #     continue
        glob_path = os.path.join(result_dir, mi_id)
        if glob_path[-1] != '/':
            glob_path = glob_path + "/" 
        mi_id_csvs = glob.glob(glob_path + "*.csv")
        if len(mi_id_csvs) != 0: 
            result_dict[mi_id]["csv_paths"] += mi_id_csvs
            
        mi_id_img = glob.glob(glob_path + "*.png")
        if len(mi_id_img) != 0: 
            result_dict[mi_id]["plot_paths"] += mi_id_img
        
img_col = "IMG"
for mi_id, v in result_dict.items():
    # if mi_id != "P0012750":
    #     continue
    subj_dir = os.path.join(save_dir, mi_id)
    if not os.path.exists(subj_dir):
        os.mkdir(subj_dir)
    v["dfs"] = [pd.read_csv(csv_path) for csv_path in v["csv_paths"]]
    img_id_list = ast.literal_eval(meta_data[meta_data['MI_ID']==mi_id]['IMG_ID_LIST'].to_list()[0])
    csv_dest_dir = os.path.join(subj_dir, "csv") 
    if not os.path.exists(csv_dest_dir):
        os.mkdir(csv_dest_dir)                            
    for indx, df in enumerate(v["dfs"]):

        type_dir = os.path.join(subj_dir, v["csv_paths"][indx].split("/")[-1].split(".csv")[0].split("_")[0])
        if not os.path.exists(type_dir):
            os.mkdir(type_dir)
            
        if "Estimate" in df.columns and "SESignificance" in df.columns:
            coef_col = "Estimate"
            if "IMG" not in df.columns:
                df["IMG"] = img_id_list
            significant_df = df.loc[df['SESignificance'] == "SIGNIFICANT"].copy()
            insignificant_df = df.loc[df['SESignificance'] == "INSIGNIFICANT"].copy()
            positive_df = significant_df.loc[significant_df["Estimate"] > 0.0].copy()
            negative_df = significant_df.loc[significant_df["Estimate"] < 0.0].copy()
            pass
        elif "corrs" in df.columns and "corr_significance" in df.columns: 
            coef_col = "corrs"
            significant_df = df.loc[df['corr_significance'] == "SIGNIFICANT"].copy()
            insignificant_df = df.loc[df['corr_significance'] == "INSIGNIFICANT"].copy()
            positive_df = significant_df.loc[significant_df["corrs"] > 0.0].copy()
            negative_df = significant_df.loc[significant_df["corrs"] <  0.0].copy()
        elif "corr" in df.columns and "corr_p_val" in df.columns: 
            v["dfs"] = pd.read_csv(v["csv_paths"][indx], index_col= 0)
            df = v["dfs"]
            coef_col = "corr"
            if "IMG" not in df.columns:
                df["IMG"] = img_id_list
            significant_df = df.loc[df['corr_p_val'] < 0.05].copy()
            insignificant_df = df.loc[df['corr_p_val'] >= 0.05].copy()
            positive_df = significant_df.loc[significant_df["corr"] > 0.0].copy()
            negative_df = significant_df.loc[significant_df["corr"] < 0.0].copy()
        else:
            continue
        
        df.to_csv(os.path.join(csv_dest_dir, f"{indx}-" + v["csv_paths"][indx].split("/")[-1]), 
                  index= None)
        
        positive_df = positive_df.sort_values([coef_col], ascending=[False])
        negative_df = negative_df.sort_values([coef_col], ascending=[True])
        positive_ranked_img = positive_df["IMG"]
        negative_ranked_img = negative_df["IMG"]
            
        positive_output_img = positive_df["IMG"] + "_" + positive_df[coef_col].round(2).astype(str)
        negative_output_img = negative_df["IMG"] + "_" + negative_df[coef_col].round(2).astype(str)
                
        positive_img_paths = [os.path.join(CROP_IMAGE_DIR, f"{mi_id}_{img_id}.jpg") for img_id in positive_ranked_img]
        negative_img_paths = [os.path.join(CROP_IMAGE_DIR, f"{mi_id}_{img_id}.jpg") for img_id in negative_ranked_img]
        neutral_img_paths = [os.path.join(CROP_IMAGE_DIR, f"{mi_id}_{img_id}.jpg") for img_id in insignificant_df["IMG"]]
        
        pos_img_dir = os.path.join(type_dir, "positive/")
        if not os.path.exists(pos_img_dir):
            os.mkdir(pos_img_dir)
        neg_img_dir = os.path.join(type_dir, "negative/")
        if not os.path.exists(neg_img_dir):
            os.mkdir(neg_img_dir)
        neutral_img_dir = os.path.join(type_dir, "neutral/")
        if not os.path.exists(neutral_img_dir):
            os.mkdir(neutral_img_dir)
                        
        output_positive_img_paths = [pos_img_dir + f"{indx+1}-{i}.png" for indx, i in enumerate(positive_output_img)]
        output_neutral_img_paths = [neutral_img_dir + i.split('/')[-1] for _, i in enumerate(neutral_img_paths)]
        output_negative_img_paths = [neg_img_dir + f"{indx+1}-{i}.png" for indx, i in enumerate(negative_output_img)]
        
        for src_img, dest_img in zip(positive_img_paths, output_positive_img_paths):
            shutil.copy2(src_img, dest_img)
            
        for src_img, dest_img in zip(neutral_img_paths, output_neutral_img_paths):
            shutil.copy2(src_img, dest_img)
            
        for src_img, dest_img in zip(negative_img_paths, output_negative_img_paths):
            shutil.copy2(src_img, dest_img)
           
    plot_save_dir = os.path.join(subj_dir, "plots")
    if not os.path.exists(plot_save_dir):
        os.mkdir(plot_save_dir)
    img_save_paths = [os.path.join(plot_save_dir, f"{indx}-" + filepath.split("/")[-1])
                      for indx, filepath in enumerate(v["plot_paths"])]
    
    for src_img, dest_img in zip(v["plot_paths"], img_save_paths):
        shutil.copy2(src_img, dest_img)
        
print(f"All results saved to {save_dir}")

All results saved to /home/liuusa_tw/twbabd_image_xai_20062024/custom_lime_results/organized-output-08-07-2024-08-43-57
